## Imports

In [1]:
import torch as t
from fancy_einsum import einsum
import einops
from dataclasses import dataclass
from pathlib import Path
import json
from tqdm import tqdm
import plotly.express as px
import numpy as np
import pandas as pd

t.manual_seed(0)

## Model

In [2]:
device = 'cuda:0'

In [3]:
@dataclass
class Config:
    d_model: int = 16
    layer_norm_eps: float = 1e-5
    d_vocab: int = 1024
    init_range: float = 0.02
    n_ctx: int = 1024
    d_head: int = 8
    d_mlp: int = 64
    n_heads: int = 2
    n_layers: int = 1

cfg = Config()
print(cfg)

Config(d_model=16, layer_norm_eps=1e-05, d_vocab=1024, init_range=0.02, n_ctx=1024, d_head=8, d_mlp=64, n_heads=2, n_layers=1)


In [4]:
class LayerNorm(t.nn.Module):
    def __init__(self, cfg: Config):
        super().__init__()
        self.cfg = cfg
        self.w = t.nn.Parameter(t.ones(cfg.d_model).to(device))
        self.b = t.nn.Parameter(t.zeros(cfg.d_model).to(device))

    def forward(self, residual):
        residual_std = (residual.var(dim=-1, keepdim=True, unbiased=False) + self.cfg.layer_norm_eps).sqrt()
        residual = (residual - residual.mean(dim=-1, keepdim=True)) / residual_std
        return residual * self.w + self.b

In [5]:
class Embed(t.nn.Module):
    def __init__(self, cfg: Config):
        super().__init__()
        self.cfg = cfg
        self.W_E = t.nn.Parameter(t.empty((cfg.d_vocab, cfg.d_model)).to(device))
        t.nn.init.normal_(self.W_E, std=self.cfg.init_range)

    def forward(self, tokens):
        return self.W_E[tokens]

In [6]:
class PosEmbed(t.nn.Module):
    def __init__(self, cfg: Config):
        super().__init__()
        self.cfg = cfg
        self.W_pos = t.nn.Parameter(t.empty((cfg.n_ctx, cfg.d_model)).to(device))
        t.nn.init.normal_(self.W_pos, std=self.cfg.init_range)

    def forward(self, tokens):
        batch, seq_len = tokens.shape
        return einops.repeat(self.W_pos[:seq_len], "seq d_m -> b seq d_m", b=batch)

In [7]:
class Attention(t.nn.Module):

    def __init__(self, cfg: Config):
        super().__init__()
        self.cfg = cfg
        self.W_Q = t.nn.Parameter(t.empty((cfg.n_heads, cfg.d_model, cfg.d_head)).to(device))
        self.W_K = t.nn.Parameter(t.empty((cfg.n_heads, cfg.d_model, cfg.d_head)).to(device))
        self.W_V = t.nn.Parameter(t.empty((cfg.n_heads, cfg.d_model, cfg.d_head)).to(device))
        self.W_O = t.nn.Parameter(t.empty((cfg.n_heads, cfg.d_head, cfg.d_model)).to(device))
        self.b_Q = t.nn.Parameter(t.zeros((cfg.n_heads, cfg.d_head)).to(device))
        self.b_K = t.nn.Parameter(t.zeros((cfg.n_heads, cfg.d_head)).to(device))
        self.b_V = t.nn.Parameter(t.zeros((cfg.n_heads, cfg.d_head)).to(device))
        self.b_O = t.nn.Parameter(t.zeros((cfg.d_model)).to(device))
        t.nn.init.normal_(self.W_Q, std=self.cfg.init_range)
        t.nn.init.normal_(self.W_K, std=self.cfg.init_range)
        t.nn.init.normal_(self.W_V, std=self.cfg.init_range)
        t.nn.init.normal_(self.W_O, std=self.cfg.init_range)

    def forward(self, normalized_resid_pre):

        q = einsum("b p d_m, h d_m d_h -> b p h d_h", normalized_resid_pre, self.W_Q) + self.b_Q
        k = einsum("b p d_m, h d_m d_h -> b p h d_h", normalized_resid_pre, self.W_K) + self.b_K
        v = einsum("b p d_m, h d_m d_h -> b p h d_h", normalized_resid_pre, self.W_V) + self.b_V

        attn_scores = einsum("b p_Q h d_h, b p_K h d_h -> b h p_Q p_K", q, k)

        attn_scores_masked = self.apply_causal_mask(attn_scores / self.cfg.d_head ** 0.5)
        attn_pattern = attn_scores_masked.softmax(-1)

        z = einsum("b p_K h d_h, b h p_Q p_K -> b p_Q h d_h", v, attn_pattern)

        attn_out = einsum("b p_Q h d_h, h d_h d_m -> b p_Q d_m", z, self.W_O) + self.b_O
        return attn_out

    def apply_causal_mask(self, attn_scores):

        all_ones = t.ones(attn_scores.size(-2), attn_scores.size(-1), device=attn_scores.device)
        mask = t.triu(all_ones, diagonal=1).bool()
        attn_scores.masked_fill_(mask, -1e5)
        return attn_scores

In [8]:
class MLP(t.nn.Module):
    def __init__(self, cfg: Config):
        super().__init__()
        self.cfg = cfg
        self.W_in = t.nn.Parameter(t.empty((cfg.d_model, cfg.d_mlp)).to(device))
        self.W_out = t.nn.Parameter(t.empty((cfg.d_mlp, cfg.d_model)).to(device))
        self.b_in = t.nn.Parameter(t.zeros((cfg.d_mlp)).to(device))
        self.b_out = t.nn.Parameter(t.zeros((cfg.d_model)).to(device))
        t.nn.init.normal_(self.W_in, std=self.cfg.init_range)
        t.nn.init.normal_(self.W_out, std=self.cfg.init_range)

    def forward(self, normalized_resid_mid):

        pre = einsum("b p d_m, d_m d_mlp -> b p d_mlp", normalized_resid_mid, self.W_in) + self.b_in
        post = t.nn.GELU()(pre)
        mlp_out = einsum("b p d_mlp, d_mlp d_m -> b p d_m", post, self.W_out) + self.b_out
        return mlp_out

In [9]:
class TransformerBlock(t.nn.Module):
    def __init__(self, cfg: Config):
        super().__init__()
        self.cfg = cfg
        self.ln1 = LayerNorm(cfg)
        self.attn = Attention(cfg)
        self.ln2 = LayerNorm(cfg)
        self.mlp = MLP(cfg)

    def forward(self, resid_pre):
        resid_mid = resid_pre + self.attn(self.ln1(resid_pre))
        resid_post = resid_mid + self.mlp(self.ln2(resid_mid))
        return resid_post

In [10]:
class Unembed(t.nn.Module):
    def __init__(self, cfg):
        super().__init__()
        self.cfg = cfg
        self.W_U = t.nn.Parameter(t.empty((cfg.d_model, cfg.d_vocab)).to(device))
        t.nn.init.normal_(self.W_U, std=self.cfg.init_range)
        self.b_U = t.nn.Parameter(t.zeros((cfg.d_vocab), requires_grad=False).to(device))

    def forward(self, normalized_resid_final):
        return einsum("b p d_m, d_m d_v -> b p d_v", normalized_resid_final, self.W_U) + self.b_U

In [11]:
class Transformer(t.nn.Module):
    def __init__(self, cfg: Config):
        super().__init__()
        self.cfg = cfg
        self.embed = Embed(cfg)
        self.pos_embed = PosEmbed(cfg)
        self.blocks = t.nn.ModuleList([TransformerBlock(cfg) for _ in range(cfg.n_layers)])
        self.ln_final = LayerNorm(cfg)
        self.unembed = Unembed(cfg)

    def forward(self, tokens):
        residual = self.embed(tokens) + self.pos_embed(tokens)
        for block in self.blocks:
            residual = block(residual)
        logits = self.unembed(self.ln_final(residual))
        return logits

## Memorised strings training

In [62]:
d_vocab = 128

model_cfg = Config(
    d_model=32,
    n_heads=4,
    d_head=8,
    d_mlp=128,
    n_layers=1,
    n_ctx=128,
    d_vocab=d_vocab
)

model = Transformer(model_cfg)

In [63]:
@dataclass
class TransformerTrainingArgs():
	batch_size = 512
	epochs = 10
	num_steps = 10000
	lr = 1e-3
	beta1 = 0.9
	beta2 = 0.999
	weight_decay = 1e-2

args = TransformerTrainingArgs()

In [64]:
num_mem_seqs = 128
mem_seq = t.randint(1, d_vocab, (num_mem_seqs, model.cfg.n_ctx))

In [65]:
SAVE_DIR = Path("/root/expl_research/mem-str-models")

In [66]:
class Trainer:
    def __init__(self, args, model):
        super().__init__()
        self.model = model
        self.args = args
        self.optimizer = t.optim.AdamW(self.model.parameters(), lr=args.lr, weight_decay=args.weight_decay)
        self.step = 0
        self.weighting = t.ones(args.batch_size).to(device)
        self.weighting[:num_mem_seqs] = (-t.linspace(-2,2,num_mem_seqs)).exp()

    def eval(self):
        tokens = t.randint(1, d_vocab, (args.batch_size, model.cfg.n_ctx)).to(device)
        starting_pos = t.randint(1, model.cfg.n_ctx, (1,)).item()
        tokens[:num_mem_seqs, starting_pos:] = mem_seq[:,:-starting_pos].to(device)
        tokens[:,0] = 0
        logits = model(tokens)
        log_probs = logits.log_softmax(dim=-1)
        correct_log_probs = -log_probs[:, :-1].gather(dim=-1, index=tokens[:, 1:].unsqueeze(-1)).squeeze(-1)
        loss = (correct_log_probs * self.weighting[:, None]).mean()
        return starting_pos, correct_log_probs, loss

    def train(self):
        try:
            for i in tqdm(range(self.args.num_steps)):
                _, _, loss = self.eval()
                loss.backward()
                self.optimizer.step()
                self.optimizer.zero_grad()
        finally:
            t.save(model.state_dict(), SAVE_DIR/("1l_learned_embed.pt"))

In [67]:
device = 'cuda:0'
trainer = Trainer(args, model)

In [115]:
trainer.train()

100%|██████████| 10000/10000 [02:16<00:00, 73.22it/s]


In [119]:
starting_pos, log_probs, loss = trainer.eval()
print(starting_pos, loss.item())

72 5.420128345489502


In [120]:
px.line(log_probs.T.cpu().detach())